<h1><center>
    DSOR 646 - Reinforcement Learning </br>
    Brandon Hosley
</center></h1>

# On-policy Monte Carlo Control (every-visit):
- [ ] Execute at least 40 runs
- [ ] Show a scatter plot (See the Lesson 9 Handout for an example of the desired scatter plot)

In [1]:
%rm -r __pycache__/
from mcc import MCC     # Local Python script
from scipy.stats.qmc import LatinHypercube
import time
import numpy as np

rm: __pycache__/: No such file or directory


In [9]:
# Testable Params
Q1 = None
eps_a = 0
eps_b = None


0


In [ ]:
# Example single experiment
num_episodes=int(0.5e3)
replications=10     # At least 10 reps per run

pol = MCC()
pol.train_on_policy(num_episodes, replications)
pol.show_results()


# Off-policy Monte Carlo Control (every-visit, weighted importance sampling):

# Compare

In [6]:
def run_experiment (run_index:int, factors:np.ndarray) -> tuple[int,float,float,float]:
    run_start_time = time.time()
    pol = MCC(factors[0],factors[1],factors[2])
    pol.train_on_policy(num_episodes,replications)
    maxETDR, maxETDRhw, meanMaxTestEETDR, maxTestHW, meanAULC, hwAULC, secs_taken = pol.get_results()
    alg_score = 0.6*(meanMaxTestEETDR-maxTestHW) + 0.4*(meanAULC-hwAULC)
    print(f"Complete experiment run {run_index} with a score of {alg_score:.2f} ({time.time() - run_start_time:.1f}s)")
    return run_index, maxETDR, maxETDRhw, meanMaxTestEETDR, maxTestHW, meanAULC, hwAULC, secs_taken